<a href="https://colab.research.google.com/github/gnapat/CNN_Imageclassified/blob/main/deep_hw01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-8f98ebeb-1264-8757-c96e-94705d3e666b)


In [1]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
print( f"Matplotlib {matplotlib.__version__}\n" )

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image

#from tensorflow.keras.applications.vgg16 import preprocess_input

Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]

NumPy 1.23.5

Matplotlib 3.7.1

TensorFlow 2.14.0
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 0 GPU(s):


In [8]:

def do_augmentation(x,y,aug):

  ret_x=[]
  ret_y=[]

  if aug == 1:
    aug_list = [Image.FLIP_LEFT_RIGHT]
  elif aug == 2:
    aug_list = [Image.ROTATE_180]
  elif aug == 3:
    #aug_list = [Image.ROTATE_90,Image.ROTATE_180,Image.ROTATE_270]
    aug_list = [Image.ROTATE_90,Image.ROTATE_90,Image.ROTATE_90]
  elif aug == 4:
    #aug_list = [Image.ROTATE_90,Image.ROTATE_180,Image.ROTATE_270,Image.FLIP_LEFT_RIGHT,Image.ROTATE_90,Image.ROTATE_180,Image.ROTATE_270]
    aug_list = [Image.ROTATE_90,Image.ROTATE_90,Image.ROTATE_90,Image.FLIP_LEFT_RIGHT,Image.ROTATE_90,Image.ROTATE_90,Image.ROTATE_90]

  cc=0
  for img in x:
    ret_x.append(img)
    ret_y.append(y[cc])

    img = tf.keras.utils.array_to_img(img)

    for iaug in aug_list:
      img = img.transpose(method=iaug)
      img = tf.keras.preprocessing.image.img_to_array(img)

      ret_x.append(img)
      ret_y.append(y[cc])

    cc += 1

  return ret_x,ret_y



def prepare_dataset(path,lables,test_size=0.2, random_state=42,split=True,augmentation=0):

  x = []
  y=[]
  images = []
  X_train = np.empty((244,244,3))
  X_test = np.array([])
  y_train = np.empty( (244,244,3))
  y_test =np.array([])

  for i in rock_label:
    folder_path=f"{path}/{rock_label[i]}"
    #print(folder_path)
    files = os.listdir(folder_path)
    for file in files:
      target_path = f"{folder_path}/{file}"
      #print(target_path)
      img =  tf.keras.preprocessing.image.load_img(target_path, target_size=(224, 224))
      img = tf.keras.preprocessing.image.img_to_array(img)
      images.append(img)
      y.append(int(i))

  X = np.array(images)
  Y = np.array(y)

  if split == True:
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
    #X_train_temp,y_train_temp = do_augmentation(X_train_temp,y_train_temp,aug=augmentation)
  else:
    X = np.array(images)
    Y = np.array(y)

  if split == True:
    return X_train_temp, X_test_temp, y_train_temp, y_test_temp
  else:
    return X,Y

In [9]:
path="drive/MyDrive/Gz/nida/dl/hw01/Rock"
#rock_label={"1":"Amphibolite","2":"Andesite","3":"Basalt","4":"Carbonatite","4":"Coal"}
#rock_label={"1":"Amphibolite","2":"Andesite"}
rock_label={"1":"Amphibolite"}

X_train,X_test,Y_train,Y_test = prepare_dataset(path=path,lables=rock_label,split=True,augmentation=3)

In [ ]:
type(X_train.shape)
X_train

In [ ]:
plt.imshow( X_train[1].astype(np.uint8) )
plt.show()

In [ ]:
img_w,img_h = 224,224     # image size of CIFAR-10
vgg_extractor = tf.keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_w, img_h, 3))

vgg_extractor.summary()

In [ ]:
tf.keras.utils.plot_model(vgg_extractor, to_file='cnn1_sequential.png', show_shapes=True, show_dtype=True, show_layer_names=False, dpi=96)

In [ ]:
# Preprocess CIFAR-10 dataset to match VGG-16's requirements
x_train_vgg = tf.keras.applications.vgg16.preprocess_input(X_train)
x_test_vgg = tf.keras.applications.vgg16.preprocess_input(X_test)

print( x_train_vgg.dtype, x_train_vgg.shape, np.min(x_train_vgg), np.max(x_train_vgg) )
print( x_test_vgg.dtype, x_test_vgg.shape, np.min(x_test_vgg), np.max(x_test_vgg) )

In [ ]:
vgg_extractor.trainable = False
for i,layer in enumerate(vgg_extractor.layers):
    print( f"Layer {i}: name = {layer.name} , trainable = {layer.trainable}" )

In [16]:
x = vgg_extractor.output

# Add our custom layer(s) to the end of the existing model
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
new_outputs = tf.keras.layers.Dense(10, activation="softmax")(x)

# Construct the main model
model = tf.keras.models.Model(inputs=vgg_extractor.inputs, outputs=new_outputs)

In [ ]:
tf.keras.utils.plot_model(model, to_file='vggoutput1.png', show_shapes=True, show_dtype=True, show_layer_names=False, dpi=96)

In [18]:
model.compile( loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"] )

In [ ]:
history = model.fit( x_train_vgg, Y_train, batch_size=128, epochs=20, verbose=1, validation_split=0.2 )

In [ ]:
results = model.evaluate(x_test_vgg, Y_test, batch_size=128)
print( f"{model.metrics_names}: {results}" )